workbook to pull coord detail from the 1hr file, and add to the processed monthly files

In [1]:
# imports
import netCDF4 as nc
import shutil

In [2]:
# files to work with
hrly_file = '/project/applied/Data/cordex-fpsc/from_escience/CNRS-CNRM/ALP-3/pr_ALP-3_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CNRM-AROME41t1_fpsconv-x2yn2-v1_1hr_199601010030-199612312330.nc'

monthly_file = '/project/applied/Data/cordex-fpsc/from_escience/CNRS-CNRM/ALP-3/historical/pr_ALP-3_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CNRM-AROME41t1_fpsconv-x2yn2-v1_mon_199601-200601.nc'

var = 'pr'

In [3]:
# open hrly file
hrly_nc = nc.Dataset(hrly_file, 'r')

In [4]:
# grab coords and grid mapping info
x_coord = hrly_nc.variables['x']
y_coord = hrly_nc.variables['y']
lc = hrly_nc.variables['Lambert_Conformal']

In [5]:
# load the monthly file
# create a copy to work with
copy_name = "/scratch/tcrocker/test_file.nc"
shutil.copy(monthly_file, copy_name)
mthly_nc = nc.Dataset(copy_name, 'r+')

In [6]:
# Add x and y dimensions to the file
mthly_nc.renameDimension('dim1', 'y')
mthly_nc.renameDimension('dim2', 'x')



In [7]:
# this is not sufficient, maybe I should use the createVariable method
#mthly_nc.variables['x'] = x_coord
#mthly_nc.variables['y'] = y_coord

# create variables
new_y = mthly_nc.createVariable('y', y_coord.datatype, y_coord.dimensions)
new_x = mthly_nc.createVariable('x', x_coord.datatype, x_coord.dimensions)

# copy metadata
for propname in y_coord.ncattrs():
    if propname == '_FillValue':
        continue
    prop = getattr(y_coord, propname)
    setattr(new_y, propname, prop)

for propname in x_coord.ncattrs():
    if propname == '_FillValue':
        continue
    prop = getattr(x_coord, propname)
    setattr(new_x, propname, prop)

# copy data
new_y[:] = y_coord[:]
new_x[:] = x_coord[:]

In [8]:
# create fixed Lambert conformal variable
new_lc = mthly_nc.createVariable('Lambert_Conformal', 'i', ())
# set properties
for propname in lc.ncattrs():
    prop = getattr(lc, propname)
    setattr(new_lc, propname, prop)

# add grid mapping attribute to file
mthly_nc.variables[var].grid_mapping = 'Lambert_Conformal'

In [9]:
print(mthly_nc.variables)

OrderedDict([('pr', <class 'netCDF4._netCDF4.Variable'>
float32 pr(time, y, x)
    standard_name: precipitation_flux
    long_name: Precipitation
    units: kg m-2 s-1
    cell_methods: time: mean month_number: year: mean
    coordinates: lat lon
    grid_mapping: Lambert_Conformal
unlimited dimensions: 
current shape = (121, 487, 587)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    axis: T
    bounds: time_bnds
    units: hours since 1949-12-01 00:00:00
    standard_name: time
    long_name: Time axis
    calendar: gregorian
    time_origin: 1949-12-01 00:00:00
unlimited dimensions: 
current shape = (121,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time_bnds', <class 'netCDF4._netCDF4.Variable'>
float64 time_bnds(time, bnds)
unlimited dimensions: 
current shape = (121, 2)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('lat', <class 'netCDF4._netCDF4.Variable'>
f

In [10]:
# tidy up
hrly_nc.close()
mthly_nc.close()